In [1]:

def df_from_redshift(sql, engine):
    with engine.connect().execution_options(autocommit=True) as conn:
        query = conn.execute(text(sql))
        df = pd.DataFrame(query.fetchall())
    return df

def df_from_s3(key, bucket,aws_access_key_id,aws_secret_access_key, s3_client=None, **args):
    if s3_client is None:
        s3_client = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    df = pd.read_parquet(io.BytesIO(obj['Body'].read()), **args)
    return df

In [2]:
def preprocesa_anio(anio):
    anio = int(anio[4:])
    return anio

## sies-matriculados

In [ ]:
bucket_name = 'cuech-siiuec-stage0-dev'
path = 'sies/matriculados/historico/2023/sies_matriculados_h_2023.parquet'

df_s3 = df_from_s3(path, bucket_name, aws_access_key_id, aws_secret_access_key)

* Matriculados por tipo de institución

In [ ]:
df_stage = df_s3[['anio','total_matriculados','clasificacion_institucion_nivel_3']].groupby(['anio','clasificacion_institucion_nivel_3'],as_index=False).sum()
df_stage

In [ ]:
df_stage['anio'] = df_stage['anio'].apply(preprocesa_anio)
df_stage

In [ ]:
sql_cubo = '''SELECT clasificacion_institucion_nivel_3,periodo,sum(total_matriculados) as total_cubo 
           FROM sies.sies_mat_h_2023
           GROUP BY clasificacion_institucion_nivel_3,periodo
           ORDER BY periodo,clasificacion_institucion_nivel_3;'''

df_rs = df_from_redshift(sql_cubo, engine_cubo)
df_rs

In [ ]:
df_stage = df_stage.sort_values(['clasificacion_institucion_nivel_3','anio'])
df_rs = df_rs.sort_values(['clasificacion_institucion_nivel_3','periodo'])

df_stage['total_cubo'] = df_rs['total_cubo'].values
df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['total_matriculados'].values

print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

* Matriculados por código de Institución

In [ ]:
df_stage = df_s3[['anio','total_matriculados','codigo_de_institucion']].groupby(['anio','codigo_de_institucion'],as_index=False).sum()
df_stage

In [ ]:
df_stage['anio'] = df_stage['anio'].apply(preprocesa_anio)
df_stage

In [ ]:
sql_cubo = '''SELECT codigo_institucion,periodo,sum(total_matriculados) as total_cubo 
           FROM sies.sies_mat_h_2023
           GROUP BY codigo_institucion,periodo
           ORDER BY periodo,codigo_institucion;'''

df_rs = df_from_redshift(sql_cubo, engine_cubo)
df_rs

In [ ]:
df_stage = df_stage.sort_values(['codigo_de_institucion','anio'])
df_rs = df_rs.sort_values(['codigo_institucion','periodo'])

df_stage['total_cubo'] = df_rs['total_cubo'].values
df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['total_matriculados'].values

print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

* Matriculados por región

In [ ]:
df_stage = df_s3[['anio','total_matriculados','region']].groupby(['anio','region'],as_index=False).sum()
df_stage

In [ ]:
df_stage['anio'] = df_stage['anio'].apply(preprocesa_anio)
df_stage

In [ ]:
sql_cubo = '''SELECT region_sede,periodo,sum(total_matriculados) as total_cubo 
           FROM sies.sies_mat_h_2023
           GROUP BY region_sede,periodo
           ORDER BY 2,1;'''

df_rs = df_from_redshift(sql_cubo, engine_cubo)
df_rs

In [ ]:
mapeo = {"Región del Libertador Bernardo O'Higgins":"Región Del Lib. B. O'Higgins",
         "Región de Valparaíso":"Región De Valparaíso",
         "Región de Atacama":"Región De Atacama",
         "Región del Bío-Bío":"Región Del Biobío",
         "Región de Coquimbo":"Región De Coquimbo",
         "Región de La Araucanía":"Región De La Araucanía",
         "Región de Ñuble":"Región De Ñuble",
         "Región de Tarapacá":"Región De Tarapacá",
         "Región de Los Ríos":"Región De Los Ríos",
         "Región del Maule":"Región Del Maule",
         "Región de Magallanes y Antártica Chilena":"Región De Magallanes Y La Antártica",
         "Región de Aysén del Gral.Ibañez del Campo":"Región De Aysén Del G. C. Ibañez",
         "Región Metropolitana de Santiago":"Región Metropolitana",
         "Región de Arica y Parinacota":"Región De Arica Y Parinacota",
         "Región de Antofagasta":"Región De Antofagasta",
         "Región de Los Lagos":"Región De Los Lagos"}

df_rs['region_sede'] = df_rs['region_sede'].map(mapeo)
df_rs

In [ ]:
df_stage = df_stage.sort_values(['region','anio'])
df_rs = df_rs.sort_values(['region_sede','periodo'])

df_stage['total_cubo'] = df_rs['total_cubo'].values
df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['total_matriculados'].values

print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

## sies-titulados

In [ ]:
bucket_name = 'cuech-siiuec-stage0-dev'
path = 'sies/titulados/historico/2023/sies_titulados_h_2023.parquet'

df_s3 = df_from_s3(path, bucket_name, aws_access_key_id, aws_secret_access_key)
df_s3.head(2)

* titulados por tipo de institución

In [ ]:
df_stage = df_s3[['anio','total_titulados','clasificacion_institucion_nivel_3']].groupby(['anio','clasificacion_institucion_nivel_3'],as_index=False).sum()
df_stage

In [ ]:
df_stage['anio'] = df_stage['anio'].apply(preprocesa_anio)
df_stage

In [ ]:
sql_cubo = '''SELECT clasificacion_institucion_nivel_3,periodo,sum(total_titulados) as total_cubo 
           FROM sies.sies_tit_h_2023
           GROUP BY clasificacion_institucion_nivel_3,periodo
           ORDER BY periodo,clasificacion_institucion_nivel_3;'''

df_rs = df_from_redshift(sql_cubo, engine_cubo)
df_rs

In [ ]:
df_stage = df_stage.sort_values(['clasificacion_institucion_nivel_3','anio'])
df_rs = df_rs.sort_values(['clasificacion_institucion_nivel_3','periodo'])

df_stage['total_cubo'] = df_rs['total_cubo'].values
df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['total_titulados'].values

print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])
#df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)].to_csv('diferencias_sies_tit_h_2023.csv',index=False)

* Titulados por código de Institución

In [ ]:
df_stage = df_s3[['anio','total_titulados','codigo_institucion']].groupby(['anio','codigo_institucion'],as_index=False).sum()
df_stage

In [ ]:
df_stage['anio'] = df_stage['anio'].apply(preprocesa_anio)
df_stage

In [ ]:
sql_cubo = '''SELECT codigo_institucion,periodo,sum(total_titulados) as total_cubo 
           FROM sies.sies_tit_h_2023
           GROUP BY codigo_institucion,periodo
           ORDER BY periodo,codigo_institucion;'''

df_rs = df_from_redshift(sql_cubo, engine_cubo)
df_rs

In [ ]:
df_stage = df_stage.sort_values(['codigo_institucion','anio'])
df_rs = df_rs.sort_values(['codigo_institucion','periodo'])

df_stage['total_cubo'] = df_rs['total_cubo'].values
df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['total_titulados'].values

print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

* Titulados por región

In [ ]:
df_stage = df_s3[['anio','total_titulados','region']].groupby(['anio','region'],as_index=False).sum()
df_stage

In [ ]:
df_stage['anio'] = df_stage['anio'].apply(preprocesa_anio)
df_stage

In [ ]:
mapeo = {"Coquimbo":"Región de Coquimbo",
         "Ñuble":"Región de Ñuble",
         "Valparaíso":"Región de Valparaíso",
         "La Araucanía":"Región de La Araucanía",
         "Tarapacá":"Región de Tarapacá",
         "Magallanes":"Región de Magallanes y Antártica Chilena",
        "Arica y Parinacota":"Región de Arica y Parinacota",
        "Los Lagos":"Región de Los Lagos",
        "Antofagasta":"Región de Antofagasta",
        "Los Ríos":"Región de Los Ríos",
        "Biobío":"Región del Bío-Bío",
        "Lib. Gral B. O'Higgins":"Región del Libertador Bernardo O'Higgins",
        "Atacama":"Región de Atacama",
        "Aysén":"Región de Aysén del Gral.Ibañez del Campo",
        "Maule":"Región del Maule",
        "Metropolitana":"Región Metropolitana de Santiago"}

df_stage['region'] = df_stage['region'].map(mapeo)
df_stage

In [ ]:
sql_cubo = '''SELECT region_sede,periodo,sum(total_titulados) as total_cubo 
           FROM sies.sies_tit_h_2023
           GROUP BY region_sede,periodo
           ORDER BY 2,1;'''

df_rs = df_from_redshift(sql_cubo, engine_cubo)
df_rs

In [ ]:
df_stage = df_stage.sort_values(['region','anio'])
df_rs = df_rs.sort_values(['region_sede','periodo'])

df_stage['total_cubo'] = df_rs['total_cubo'].values
df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['total_titulados'].values

print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

## sies-oferta-académica

In [ ]:
bucket_name = 'cuech-siiuec-stage0-dev'
path = 'sies/oferta-academica/historico/2024/sies_oferta_academica_h_2024.parquet'

df_s3 = df_from_s3(path, bucket_name, aws_access_key_id, aws_secret_access_key)
df_s3.head(2)

* cantidad programas por tipo de institucion

In [ ]:
df_stage = df_s3[['anio','codigo_unico','tipo_institucion_3']].groupby(['anio','tipo_institucion_3'],as_index=False).count()
df_stage

In [ ]:
df_stage['anio'] = df_stage['anio'].apply(preprocesa_anio)
df_stage

In [ ]:
sql_cubo = '''SELECT clasificacion_institucion_nivel_3,periodo,count(cod_unico) as total_cubo 
           FROM sies.sies_oa_h_2024
           GROUP BY clasificacion_institucion_nivel_3,periodo
           ORDER BY periodo,clasificacion_institucion_nivel_3;'''

df_rs = df_from_redshift(sql_cubo, engine_cubo)
df_rs

In [ ]:
df_stage = df_stage.sort_values(['tipo_institucion_3','anio'])
df_rs = df_rs.sort_values(['clasificacion_institucion_nivel_3','periodo'])

df_stage['total_cubo'] = df_rs['total_cubo'].values
df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['codigo_unico'].values

print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

* Cantidad de programas por código de institución

In [ ]:
df_stage = df_s3[['anio','codigo_unico','codigo_ies']].groupby(['anio','codigo_ies'],as_index=False).count()
df_stage

In [ ]:
df_stage['anio'] = df_stage['anio'].apply(preprocesa_anio)
df_stage

In [ ]:
sql_cubo = '''SELECT codigo_institucion,periodo,count(cod_unico) as total_cubo 
           FROM sies.sies_oa_h_2024
           GROUP BY codigo_institucion,periodo
           ORDER BY periodo,codigo_institucion;'''

df_rs = df_from_redshift(sql_cubo, engine_cubo)
df_rs

In [ ]:
df_stage = df_stage.sort_values(['codigo_ies','anio'])
df_rs = df_rs.sort_values(['codigo_institucion','periodo'])

df_stage['total_cubo'] = df_rs['total_cubo'].values
df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['codigo_unico'].values

print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

* Cantidad de programas por región

In [ ]:
df_stage = df_s3[['anio','codigo_unico','region_sede']].groupby(['anio','region_sede'],as_index=False).count()
df_stage

In [ ]:
df_stage['anio'] = df_stage['anio'].apply(preprocesa_anio)
df_stage

In [ ]:
mapeo = {"REGION DE COQUIMBO":"Región de Coquimbo",
         "REGION DE ÑUBLE":"Región de Ñuble",
         "REGION DE VALPARAISO":"Región de Valparaíso",
         "REGION DE LA ARAUCANIA":"Región de La Araucanía",
         "REGION DE TARAPACA":"Región de Tarapacá",
         "REGION DE MAGALLANES Y DE LA ANTARTICA CHILENA":"Región de Magallanes y Antártica Chilena",
        "REGION DE ARICA Y PARINACOTA":"Región de Arica y Parinacota",
        "REGION DE LOS LAGOS":"Región de Los Lagos",
        "REGION DE ANTOFAGASTA":"Región de Antofagasta",
        "REGION DE LOS RIOS":"Región de Los Ríos",
        "REGION DEL BIOBIO":"Región del Bío-Bío",
        "REGION DEL LIBERTADOR BERNARDO OHIGGINS":"Región del Libertador Bernardo O'Higgins",
        "REGION DE ATACAMA":"Región de Atacama",
        "REGION DE AYSEN DEL GENERAL CARLOS IBAÑEZ DEL CAMPO":"Región de Aysén del Gral.Ibañez del Campo",
        "REGION DEL MAULE":"Región del Maule",
        "REGION METROPOLITANA":"Región Metropolitana de Santiago"}

df_stage['region_sede'] = df_stage['region_sede'].map(mapeo)
df_stage

In [ ]:
sql_cubo = '''SELECT region_sede,periodo,count(cod_unico) as total_cubo 
           FROM sies.sies_oa_h_2024
           GROUP BY region_sede,periodo
           ORDER BY periodo,region_sede;'''

df_rs = df_from_redshift(sql_cubo, engine_cubo)
df_rs

In [ ]:
df_stage = df_stage.sort_values(['region_sede','anio'])
df_rs = df_rs.sort_values(['region_sede','periodo'])

df_stage['total_cubo'] = df_rs['total_cubo'].values
df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['codigo_unico'].values

print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])